In [36]:
from math import *
import numpy as np

# Formulas

## Aula 20

#### Forças Internas
- M – Momento Fletor
- V – Cortante
- F - Normal


<img src="img/MVF.png" alt="Diagrama MVD" width="300" height="275">


## Aula 21

### Fator de Segurança
$F_{s} = \frac{\sigma_{rup}}{\sigma}$
- $F_{s}$ = Fator de Segurança
- $\sigma_{rup}$ = Tensão de Ruptura
- $\sigma$ = Tensão Atuante


## Aula 23

### Matriz de Rigidez: Elemento de Barra/Treliça
$\sigma = \frac{F}{A}$
- $\sigma$ = Tensão
- $F$ = Força
- $A$ = Área

$u = \frac{FL}{EA}$
- $u$ = Deslocamento
- $F$ = Força Externa Aplicada
- $L$ = Comprimento da Barra
- $E$ = Módulo de Elasticidade Longitudinal do Material da Barra
- $A$ = Área da Seção Transversal da Barra

### Elemento de Barra/Treliça Plana com 2 Nós
- $\delta = \frac{Pl}{AE}$

### Modelagem por Elementos Finitos
- $[K] \cdot \{u\} = \{F\}$
- $[K]$ = Matriz de Rigidez do Elemento GLOBAL
- $\{u\}$ = Vetor de Deslocamentos NODAL, no sistema GLOBAL
- $\{F\}$ = Vetor de Carga NODAL, no sistema GLOBAL

### Matriz de Rigidez do Elemento no Sistema GLOBAL
$[K_e] = \frac{EA}{l} \begin{bmatrix} c^2 & cs & -c^2 & -cs \\ cs & s^2 & -cs & -s^2 \\ -c^2 & -cs & c^2 & cs \\ -cs & -s^2 & cs & s^2 \end{bmatrix}$


<img src="img/RPE.jpg" alt="Diagrama RPE" width="300" height="275">
<!-- <img src="img/desenho.png" alt="Desenho RPE" width="100" height="175"> -->

<!-- ![desenho](img/desenho.png) -->

# Codigo organizado

In [37]:
class Estrutura:
    def __init__(self, nos, incidencia, propriedades):
        self.nos = nos
        self.incidencia = incidencia
        self.propriedades = propriedades
        self.gdl = self._calcular_gdl()

    def _calcular_gdl(self):
        num_nos = len(self.nos)
        gdl = np.zeros((num_nos, 2), dtype=int)
        for i in range(num_nos):
            gdl[i] = [2 * i, 2 * i + 1]
        return gdl

    def matriz_rigidez_elemento(self, x1, y1, x2, y2):
        A = self.propriedades['A']
        E = self.propriedades['E']
        L = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
        c = (x2 - x1) / L
        s = (y2 - y1) / L
        k = (A * E / L) * np.array([
            [c*c, c*s, -c*c, -c*s],
            [c*s, s*s, -c*s, -s*s],
            [-c*c, -c*s, c*c, c*s],
            [-c*s, -s*s, c*s, s*s]
        ])
        return k

In [38]:
class AnaliseEstrutural:
    def __init__(self, estrutura):
        self.estrutura = estrutura
        self.K_global = self._montar_matriz_rigidez_global()

    def _montar_matriz_rigidez_global(self):
        num_gdl = 2 * len(self.estrutura.nos)
        K_global = np.zeros((num_gdl, num_gdl))
        for elem in self.estrutura.incidencia:
            i, j = elem
            x1, y1 = self.estrutura.nos[i]
            x2, y2 = self.estrutura.nos[j]
            k = self.estrutura.matriz_rigidez_elemento(x1, y1, x2, y2)
            gdl_elem = np.hstack((self.estrutura.gdl[i], self.estrutura.gdl[j]))
            for a in range(4):
                for b in range(4):
                    K_global[gdl_elem[a], gdl_elem[b]] += k[a, b]
        return K_global

    def aplicar_condicoes_contorno(self, restricoes, F):
        for i, g in enumerate(restricoes['gdl']):
            self.K_global[g, :] = 0
            self.K_global[g, g] = 1
            F[g] = restricoes['valores'][i]
        return F

    def gauss_seidel(self, A, b, x0, tol=1e-10, max_iter=1000):
        n = len(b)
        x = x0.copy()
        for k in range(max_iter):
            x_new = np.zeros_like(x)
            for i in range(n):
                s1 = np.dot(A[i, :i], x_new[:i])
                s2 = np.dot(A[i, i + 1:], x[i + 1:])
                x_new[i] = (b[i] - s1 - s2) / A[i, i]
            if np.allclose(x, x_new, atol=tol):
                break
            x = x_new
        return x

    def resolver_sistema(self, F, restricoes):
        F = self.aplicar_condicoes_contorno(restricoes, F)
        x0 = np.zeros(len(F))
        deslocamentos = self.gauss_seidel(self.K_global, F, x0)
        return deslocamentos

    def calcular_tensoes(self, deslocamentos):
        tensoes = []
        for elem in self.estrutura.incidencia:
            i, j = elem
            x1, y1 = self.estrutura.nos[i]
            x2, y2 = self.estrutura.nos[j]
            L = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
            c = (x2 - x1) / L
            s = (y2 - y1) / L
            gdl_elem = np.hstack((self.estrutura.gdl[i], self.estrutura.gdl[j]))
            u_elem = deslocamentos[gdl_elem]
            deformacao = (1 / L) * np.dot([-c, -s, c, s], u_elem)
            tensao = self.estrutura.propriedades['E'] * deformacao
            tensoes.append(tensao)
        return tensoes

    def calcular_reacoes(self, deslocamentos, F):
        return np.dot(self.K_global, deslocamentos) - F

    def verificar_falha_tensao(self, tensoes):
        falha_tensao = []
        for tensao in tensoes:
            if tensao > self.estrutura.propriedades['T_rup_tensao']:
                falha_tensao.append("Falha por tração")
            elif tensao < -self.estrutura.propriedades['T_rup_compressao']:
                falha_tensao.append("Falha por compressão")
            else:
                falha_tensao.append("Sem falha")
        return falha_tensao

    def verificar_falha_flambagem(self, tensoes):
        falha_flambagem = []
        I = (self.estrutura.propriedades['A']**2) / 12  # Momento de inércia para seção retangular
        for tensao, elem in zip(tensoes, self.estrutura.incidencia):
            if tensao < 0:  # Só considerar elementos em compressão
                i, j = elem
                L = np.sqrt((self.estrutura.nos[j, 0] - self.estrutura.nos[i, 0])**2 + (self.estrutura.nos[j, 1] - self.estrutura.nos[i, 1])**2)
                P_cr = (np.pi**2 * self.estrutura.propriedades['E'] * I) / (L**2)  # Carga crítica de flambagem
                if abs(tensao * self.estrutura.propriedades['A']) > P_cr:
                    falha_flambagem.append("Falha por flambagem")
                else:
                    falha_flambagem.append("Sem falha por flambagem")
            else:
                falha_flambagem.append("Sem falha por flambagem")
        return falha_flambagem


In [39]:
# Exemplo de uso
L = 1.0  # Comprimento de cada segmento
nos = np.array([
    [0, 0], [L, 0], [2*L, 0], [3*L, 0], [4*L, 0], [5*L, 0], [6*L, 0], [7*L, 0], [8*L, 0],  # Nós inferiores
    [L, L], [2*L, L], [3*L, L], [4*L, L], [5*L, L], [6*L, L], [7*L, L]  # Nós superiores
])
incidencia = np.array([
    [0, 1], [1, 2], [2, 3], [3, 4], [4, 5], [5, 6], [6, 7], [7, 8],  # Elementos horizontais inferiores
    [0, 9], [1, 9], [1, 10], [2, 10], [2, 11], [3, 11], [3, 12], [4, 12], [4, 13], [5, 13], [5, 14], [6, 14], [6, 15], [7, 15],  # Elementos diagonais
    [9, 10], [10, 11], [11, 12], [12, 13], [13, 14], [14, 15]  # Elementos horizontais superiores
])
propriedades = {
    'A': 6e-4,  # Área da seção transversal em m²
    'E': 200e9,  # Módulo de elasticidade em Pa (200 GPa)
    'T_rup_tensao': 400e6,  # Tensão de ruptura a tração em Pa
    'T_rup_compressao': 250e6,  # Tensão de ruptura a compressão em Pa
    'comprimento': L  # Comprimento dos elementos
}
restricoes = {
    'gdl': [0, 1, 17],  # Graus de liberdade restritos (nós 0 e 8)
    'valores': [0, 0, 0]  # Valores de deslocamento para os graus de liberdade restritos
}
cargas = {
    'gdl': [21, 23, 25],  # Graus de liberdade verticais dos nós 11, 12 e 13
    'valores': [-100, -100, -100]  # Forças aplicadas de 1000 N cada, para baixo (negativo)
}

# Inicialização da estrutura e análise
estrutura = Estrutura(nos, incidencia, propriedades)
analise = AnaliseEstrutural(estrutura)

# Montagem do vetor de carga global
F = np.zeros(32)
for i, g in enumerate(cargas['gdl']):
    F[g] = cargas['valores'][i]

# Solução do sistema de equações
deslocamentos = analise.resolver_sistema(F, restricoes)

# Pós-processamento: cálculo das tensões e reações de apoio
tensoes = analise.calcular_tensoes(deslocamentos)
reacoes = analise.calcular_reacoes(deslocamentos, F)
falha_tensao = analise.verificar_falha_tensao(tensoes)
falha_flambagem = analise.verificar_falha_flambagem(tensoes)

# # Resultados
# print("Deslocamentos nos nós (em m):")
# print(deslocamentos)
# print("\nTensões em cada elemento (em Pa):")
# print(tensoes)
# print("\nReações de apoio nos nós com restrição (em N):")
# print(reacoes[restricoes['gdl']])
# print("\nAnálise de falha por tensão:")
# print(falha_tensao)
# print("\nAnálise de falha por flambagem:")
# print(falha_flambagem)


In [40]:
A = 1
E = 1
def matriz_rigidez_elemento(x1, y1, x2, y2):
    L = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    c = (x2 - x1) / L
    s = (y2 - y1) / L
    k = (A * E / L) * np.array([
        [c*c, c*s, -c*c, -c*s],
        [c*s, s*s, -c*s, -s*s],
        [-c*c, -c*s, c*c, c*s],
        [-c*s, -s*s, c*s, s*s]
    ])
    return k

def angulo_matriz_rigidez_elemento(angulo):
    angulo = np.deg2rad(angulo)
    c = np.cos(angulo)
    s = np.sin(angulo)
    k = (A * E) * np.array([
        [c*c, c*s, -c*c, -c*s],
        [c*s, s*s, -c*s, -s*s],
        [-c*c, -c*s, c*c, c*s],
        [-c*s, -s*s, c*s, s*s]
    ])
    return k

# Codigo Miele



In [41]:
# Matriz K com condições de contorno aplicadas
# Matriz F com condições de contorno aplicadas
def met_it(K, F):
    delta = 10**(-10)
    lista_u = [0, 0, 0]
    atinjiu = False
    while atinjiu == False:
        x1 = (F[0] - K[0][1] * lista_u[1] - K[0][2] * lista_u[2]) / K[0][0]
        x2 = (F[1] - K[1][0] * lista_u[0] - K[1][2] * lista_u[2]) / K[1][1]
        x3 = (F[2] - K[2][0] * lista_u[0] - K[2][1] * lista_u[1]) / K[2][2]
        if lista_u[0] - x1 < delta and lista_u[1] - x2 < delta and lista_u[2] - x3 < delta: 
            atinjiu = True
        
        lista_u = [x1, x2, x3]
    return lista_u


def mult_K (K, val): # Multiplica a matriz K por EA/L ou 10^x
    for i in range(3):
        for j in range(3):
            K[i][j] = K[i][j] * val
    return K


K = [[1.59, -0.4, -0.54],
    [-0.4, 1.7, 0.4],
    [-0.54, 0.4, 0.54]]
F = [0, 150, -100]

K = mult_K(K, 10**8)
# print(met_it(K, F))

#### Elementos Finitos

In [42]:
def montar_matriz_local(angulo, E, A, L):
    # Convertendo o ângulo de graus para radianos
    angulo_rad = radians(angulo)
    
    # Calculando seno e cosseno
    s = round(sin(angulo_rad), 2)
    c = round(cos(angulo_rad), 2)
    cs = c * s
    s2 = s ** 2
    c2 = c ** 2
    # Construindo a matriz de rigidez
    mat = np.array([
        [c2, cs, -c2, -cs],
        [cs, s2, -cs, -s2],
        [-c2, -cs, c2, cs],
        [-cs, -s2, cs, s2]
    ])
    # Multiplicando todos os elementos por (E*A)/L
    mat *= (E * A) / L
    return mat

# Exemplo de uso
ang1, ang2, ang3, E, A, L1, L2, L3 = 90, 0, 53, 210*10**9, 2*10**(-4), 0.4, 0.3, 0.5
K1 = montar_matriz_local(ang1, E, A, L1)
K2 = montar_matriz_local(ang2, E, A, L2)
K3 = montar_matriz_local(ang3, E, A, L3)
# for linha in K1:
#     print(linha)
# print("\n")
# for linha in K2:
#     print(linha)
# print("\n")
# for linha in K3:
#     print(linha)


In [43]:
def montar_matriz_global(matrizes_locais, num_nos):
    # Cada nó tem 2 graus de liberdade, assumindo problemas planos.
    K_global = np.zeros((2 * num_nos, 2 * num_nos))
    for matriz_local, nos in matrizes_locais:
        indices = []
        for no in nos:
            indices.extend([2 * (no - 1), 2 * (no - 1) + 1])  # Graus de liberdade para x e y
        # Adicionando a matriz local na matriz global
        for i in range(4):
            for j in range(4):
                K_global[indices[i], indices[j]] += matriz_local[i, j]
    return K_global

# Exemplo de uso
matrizes_locais = [
    (K1, [1, 2]),  
    (K2, [2, 3]), 
    (K3, [3, 1])
]
num_nos = 3

K_global = montar_matriz_global(matrizes_locais, num_nos)
# print(K_global)

In [44]:
def deformacao(L, angulo, lista_u):
    angulo_rad = radians(angulo)
    s = round(sin(angulo_rad), 0)
    c = round(cos(angulo_rad), 0)
    lista_cs = [-c, -s, c, s]

    return (lista_cs[0] * lista_u[0] + lista_cs[1] * lista_u[1] + lista_cs[2] * lista_u[2] + lista_cs[3] * lista_u[3]) / L

# print(deformacao(0.4, 90, [0, -9.52e-7, 0, 0]))

def tensao(E, L, angulo, lista_u):
    angulo_rad = radians(angulo)
    s = round(sin(angulo_rad), 0)
    c = round(cos(angulo_rad), 0)
    lista_cs = [-c, -s, c, s]

    return E * (lista_cs[0] * lista_u[0] + lista_cs[1] * lista_u[1] + lista_cs[2] * lista_u[2] + lista_cs[3] * lista_u[3]) / L



# Clear all variables


In [45]:
%reset -f

[[159000000.0, -40000000.0, -54000000.0], [-40000000.0, 170000000.0, 40000000.0], [-54000000.0, 40000000.0, 54000000.0]]


NameError: name 'K' is not defined

---
# Faça a Prova Aqui 
# Downward arrow 